<a href="https://colab.research.google.com/github/jalevano/tfm_uoc_datascience/blob/main/03_Analisis_Fase_2E_Bloque_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.insert(0, '/content/drive/MyDrive/TFM/3_Analisis/fase2e_visualizaciones')
from analisis_fase_2e_config import inicializar_fase2e
estado = inicializar_fase2e()

Entorno detectado: colab
Ruta base: /content/drive/MyDrive/TFM
[01:54:44] INFO - ============================================================


INFO:Fase2E_Init:============================================================


[01:54:44] INFO - INICIALIZANDO FASE 2E - VISUALIZACIONES


INFO:Fase2E_Init:INICIALIZANDO FASE 2E - VISUALIZACIONES


[01:54:44] INFO - ============================================================


...

INFO:Fase2E_Init:  anova_por_modelo: OK


[01:54:46] INFO -   indice_maestro: OK


INFO:Fase2E_Init:  indice_maestro: OK


[01:54:46] INFO - Archivos disponibles: 10/10


INFO:Fase2E_Init:Archivos disponibles: 10/10


In [ ]:
# -*- coding: utf-8 -*-
"""
================================================================================
FASE 2E - BLOQUE 4: CATALOGO FOTOGRAFICO
================================================================================

Trabajo Fin de Master - Evaluacion Comparativa de Tecnicas de Segmentacion
                        en Fotografia de Retrato

Autor: Jesus L.
Universidad: Universitat Oberta de Catalunya (UOC)
Master: Data Science
Fecha: Diciembre 2025

Descripcion:
    Generacion de 4 visualizaciones de catalogo fotografico:
    - Mosaico del dataset agrupado por clusters de dificultad
    - Panel de distribucion de caracteristicas EXIF
    - Scatter de caracteristicas vs dificultad
    - Galeria anotada con metricas Shapely

Correcciones aplicadas:
    - squeeze=False en plt.subplots para arrays 2D garantizados
    - axes.flatten() para acceso seguro por indice
    - Validacion de mascaras 1D antes de redimensionar
    - Manejo robusto de datos faltantes

Uso:
    from analisis_fase_2e_bloque4 import ejecutar_bloque4
    resultados = ejecutar_bloque4()
================================================================================
"""

'\n================================================================================\nFASE 2E - BLOQUE 4: CATALOGO FOTOGRAFICO\n================================================================================\n\nTrabajo Fin de Master - Evaluacion Comparativa de Tecnicas de Segmentacion\n                        en Fotografia de Retrato\n\nAutor: Jesus L.\nUniversidad: Universitat Oberta de Catalunya (UOC)\nMaster: Data Science\nFecha: Diciembre 2025\n\nDescripcion:\n    Generacion de 4 visualizaciones de catalogo fotografico:\n    - Mosaico del dataset agrupado por clusters de dificultad\n    - Panel de distribucion de caracteristicas EXIF\n    - Scatter de caracteristicas vs dificultad\n    - Galeria anotada con metricas Shapely\n\nCorrecciones aplicadas:\n    - squeeze=False en plt.subplots para arrays 2D garantizados\n    - axes.flatten() para acceso seguro por indice\n    - Validacion de mascaras 1D antes de redimensionar\n    - Manejo robusto de datos faltantes\n\nUso:\n    from ana

In [ ]:

# =============================================================================
# IMPORTS
# =============================================================================

import sys
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.patches import Patch, Rectangle
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from pathlib import Path
from typing import Dict, List, Tuple, Optional
from PIL import Image

# Importar configuracion compartida
try:
    from analisis_fase_2e_config import (
        RUTAS, ARCHIVOS, FIGSIZE,
        PALETTE_MODELS, ORDEN_MODELOS, NOMBRES_MODELOS,
        PALETTE_OVERLAY, FOTOS_SELECCIONADAS,
        configurar_logging, aplicar_estilo, guardar_figura,
        agregar_columna_modelo, extraer_modelo_de_config
    )
except ImportError:
    print("Ejecutando importacion alternativa de configuracion...")
    exec(open('/content/drive/MyDrive/TFM/3_Analisis/fase2e_visualizaciones/analisis_fase_2e_config.py').read())

# =============================================================================
# CONSTANTES
# =============================================================================

TEXTOS = {
    'titulo_mosaico': 'Catalogo del Dataset por Nivel de Dificultad',
    'titulo_exif': 'Distribucion de Caracteristicas Fotograficas (EXIF)',
    'titulo_scatter': 'Caracteristicas Fotograficas vs Dificultad de Segmentacion',
    'titulo_galeria': 'Galeria Anotada - Fotografias Seleccionadas',
    'cluster_facil': 'Facil',
    'cluster_medio': 'Medio',
    'cluster_dificil': 'Dificil',
    'iou_promedio': 'IoU Promedio',
    'varianza': 'Varianza',
}

# Colores para clusters de dificultad
COLORES_CLUSTER = {
    'facil': '#2ECC71',      # Verde
    'medio': '#F39C12',      # Naranja
    'dificil': '#E74C3C',    # Rojo
    0: '#2ECC71',
    1: '#F39C12',
    2: '#E74C3C',
}

# Caracteristicas EXIF a visualizar
CARACTERISTICAS_EXIF = [
    'apertura_fnumber',
    'iso',
    'tiempo_exposicion_segundos',
    'distancia_focal',
    'brillo_promedio',
    'contraste_rms',
    'saturacion_media',
    'nitidez_laplacian'
]

NOMBRES_CARACTERISTICAS = {
    'apertura_fnumber': 'Apertura (f/)',
    'iso': 'ISO',
    'tiempo_exposicion_segundos': 'Tiempo Exp. (s)',
    'distancia_focal': 'Dist. Focal (mm)',
    'brillo_promedio': 'Brillo Medio',
    'contraste_rms': 'Contraste RMS',
    'saturacion_media': 'Saturacion Media',
    'nitidez_laplacian': 'Nitidez (Laplacian)'
}


# =============================================================================
# FUNCIONES AUXILIARES
# =============================================================================

def cargar_imagen_thumbnail(foto_id: str, ruta_imagenes: Path, size: Tuple[int, int] = (200, 200)) -> Optional[np.ndarray]:
    """
    Carga imagen como thumbnail redimensionado.

    Args:
        foto_id: Identificador de la fotografia
        ruta_imagenes: Ruta al directorio de imagenes
        size: Tamano del thumbnail (ancho, alto)

    Returns:
        Array numpy RGB del thumbnail o None
    """
    extensiones = ['.jpg', '.jpeg', '.JPG', '.JPEG', '.png', '.PNG']

    for ext in extensiones:
        ruta = ruta_imagenes / f"{foto_id}{ext}"
        if ruta.exists():
            try:
                img = Image.open(ruta).convert('RGB')
                img.thumbnail(size, Image.LANCZOS)
                return np.array(img)
            except Exception as e:
                continue
    return None


def calcular_dificultad_foto(df: pd.DataFrame, col_foto: str) -> pd.DataFrame:
    """
    Calcula metricas de dificultad por fotografia.

    Args:
        df: DataFrame con metricas
        col_foto: Nombre de la columna de foto

    Returns:
        DataFrame con estadisticas por foto
    """
    # Obtener mejor IoU por modelo para cada foto
    df_best = df.loc[df.groupby([col_foto, 'modelo_norm'])['iou'].idxmax()]

    # Calcular estadisticas por foto
    stats = df_best.groupby(col_foto).agg({
        'iou': ['mean', 'std', 'min', 'max', 'count']
    }).reset_index()

    stats.columns = [col_foto, 'iou_mean', 'iou_std', 'iou_min', 'iou_max', 'n_modelos']

    # Calcular rango y coeficiente de variacion
    stats['iou_rango'] = stats['iou_max'] - stats['iou_min']
    stats['cv'] = stats['iou_std'] / stats['iou_mean']

    # Asignar cluster de dificultad basado en IoU medio
    stats = stats.sort_values('iou_mean', ascending=False).reset_index(drop=True)
    n = len(stats)

    def asignar_cluster(idx):
        if idx < n // 3:
            return 'facil'
        elif idx < 2 * n // 3:
            return 'medio'
        else:
            return 'dificil'

    stats['cluster'] = [asignar_cluster(i) for i in range(n)]

    return stats


def extraer_caracteristicas_exif(indice_maestro: Dict, foto_id: str) -> Dict:
    """
    Extrae caracteristicas EXIF del indice maestro.

    Args:
        indice_maestro: Diccionario del indice maestro
        foto_id: Identificador de la foto

    Returns:
        Diccionario con caracteristicas
    """
    if foto_id not in indice_maestro:
        return {}

    foto_info = indice_maestro[foto_id]
    resumen = foto_info.get('resumen_caracteristicas', {})

    caracteristicas = {}

    # EXIF
    exif = resumen.get('exif', {})
    caracteristicas['apertura_fnumber'] = exif.get('apertura_fnumber')
    caracteristicas['iso'] = exif.get('iso')
    caracteristicas['tiempo_exposicion_segundos'] = exif.get('tiempo_exposicion_segundos')
    caracteristicas['distancia_focal'] = exif.get('distancia_focal')

    # Calidad
    calidad = resumen.get('calidad', {})
    caracteristicas['brillo_promedio'] = calidad.get('exposicion_brillo_medio')
    caracteristicas['contraste_rms'] = calidad.get('contraste_rms')
    caracteristicas['nitidez_laplacian'] = calidad.get('nitidez_laplacian')

    # Color
    color = resumen.get('color', {})
    caracteristicas['saturacion_media'] = color.get('saturacion_mean')

    return caracteristicas


# =============================================================================
# VISUALIZACION 28: MOSAICO DEL DATASET POR CLUSTERS
# =============================================================================

def generar_mosaico_clusters(
    df: pd.DataFrame,
    ruta_imagenes: Path,
    output_dir: Path,
    indice_maestro: Dict = None,
    logger=None
) -> Optional[plt.Figure]:
    """
    Genera mosaico del dataset agrupado por clusters de dificultad.

    Muestra thumbnails de todas las fotografias organizadas en filas
    segun su nivel de dificultad (facil, medio, dificil).

    Args:
        df: DataFrame con metricas
        ruta_imagenes: Ruta a imagenes originales
        output_dir: Directorio de salida
        indice_maestro: Diccionario del indice maestro
        logger: Logger opcional

    Returns:
        Figura matplotlib o None
    """
    if logger:
        logger.info("  Generando mosaico por clusters...")

    col_foto = 'codigo_foto' if 'codigo_foto' in df.columns else 'foto_id'

    # Calcular dificultad
    stats = calcular_dificultad_foto(df, col_foto)

    # Agrupar por cluster
    clusters = {
        'facil': stats[stats['cluster'] == 'facil'][col_foto].tolist(),
        'medio': stats[stats['cluster'] == 'medio'][col_foto].tolist(),
        'dificil': stats[stats['cluster'] == 'dificil'][col_foto].tolist()
    }

    # Calcular layout
    max_por_fila = max(len(v) for v in clusters.values())
    n_filas = 3

    fig, axes = plt.subplots(n_filas, max_por_fila,
                              figsize=(2.5 * max_por_fila, 8),
                              squeeze=False)

    cluster_orden = ['facil', 'medio', 'dificil']
    cluster_labels = [TEXTOS['cluster_facil'], TEXTOS['cluster_medio'], TEXTOS['cluster_dificil']]

    for row_idx, (cluster_key, cluster_label) in enumerate(zip(cluster_orden, cluster_labels)):
        fotos = clusters[cluster_key]
        color = COLORES_CLUSTER[cluster_key]

        for col_idx in range(max_por_fila):
            ax = axes[row_idx, col_idx]

            if col_idx < len(fotos):
                foto_id = fotos[col_idx]
                thumbnail = cargar_imagen_thumbnail(foto_id, ruta_imagenes, size=(200, 200))

                if thumbnail is not None:
                    ax.imshow(thumbnail)

                    # Obtener IoU de esta foto
                    iou_mean = stats[stats[col_foto] == foto_id]['iou_mean'].values[0]

                    # Borde coloreado
                    for spine in ax.spines.values():
                        spine.set_edgecolor(color)
                        spine.set_linewidth(3)

                    ax.set_title(f'{foto_id}\nIoU: {iou_mean:.3f}', fontsize=8)
                else:
                    ax.text(0.5, 0.5, foto_id, ha='center', va='center', fontsize=8)
                    ax.set_facecolor('#f0f0f0')
            else:
                ax.axis('off')

            ax.set_xticks([])
            ax.set_yticks([])

        # Etiqueta de fila
        axes[row_idx, 0].set_ylabel(cluster_label, fontsize=12, fontweight='bold',
                                     color=color, rotation=0, labelpad=50, va='center')

    fig.suptitle(TEXTOS['titulo_mosaico'], fontsize=14, fontweight='bold', y=0.98)

    # Leyenda
    legend_elements = [
        Patch(facecolor=COLORES_CLUSTER['facil'], label=f"{TEXTOS['cluster_facil']} (IoU > 0.85)"),
        Patch(facecolor=COLORES_CLUSTER['medio'], label=f"{TEXTOS['cluster_medio']} (0.75 < IoU < 0.85)"),
        Patch(facecolor=COLORES_CLUSTER['dificil'], label=f"{TEXTOS['cluster_dificil']} (IoU < 0.75)")
    ]
    fig.legend(handles=legend_elements, loc='lower center', ncol=3, fontsize=9,
               bbox_to_anchor=(0.5, 0.01))

    plt.tight_layout(rect=[0.05, 0.05, 1, 0.95])

    guardar_figura(fig, 'viz_28_mosaico_clusters', output_dir, logger=logger)

    if logger:
        logger.info(f"    Clusters: Facil={len(clusters['facil'])}, "
                   f"Medio={len(clusters['medio'])}, Dificil={len(clusters['dificil'])}")

    return fig


# =============================================================================
# VISUALIZACION 29: PANEL DE CARACTERISTICAS EXIF
# =============================================================================

def generar_panel_exif(
    df: pd.DataFrame,
    indice_maestro: Dict,
    output_dir: Path,
    logger=None
) -> Optional[plt.Figure]:
    """
    Genera panel con distribucion de caracteristicas EXIF.

    Muestra histogramas de las principales caracteristicas fotograficas
    coloreados por cluster de dificultad.

    Args:
        df: DataFrame con metricas
        indice_maestro: Diccionario del indice maestro
        output_dir: Directorio de salida
        logger: Logger opcional

    Returns:
        Figura matplotlib o None
    """
    if logger:
        logger.info("  Generando panel EXIF...")

    col_foto = 'codigo_foto' if 'codigo_foto' in df.columns else 'foto_id'

    # Calcular dificultad
    stats = calcular_dificultad_foto(df, col_foto)

    # Extraer caracteristicas EXIF para cada foto
    datos_exif = []
    for foto_id in stats[col_foto]:
        caract = extraer_caracteristicas_exif(indice_maestro, foto_id)
        caract['foto_id'] = foto_id
        caract['cluster'] = stats[stats[col_foto] == foto_id]['cluster'].values[0]
        caract['iou_mean'] = stats[stats[col_foto] == foto_id]['iou_mean'].values[0]
        datos_exif.append(caract)

    df_exif = pd.DataFrame(datos_exif)

    # Convertir columnas numericas
    for col in CARACTERISTICAS_EXIF:
        if col in df_exif.columns:
            df_exif[col] = pd.to_numeric(df_exif[col], errors='coerce')

    # Crear figura con subplots
    n_caract = len(CARACTERISTICAS_EXIF)
    ncols = 4
    nrows = (n_caract + ncols - 1) // ncols

    fig, axes = plt.subplots(nrows, ncols, figsize=(14, 3.5 * nrows), squeeze=False)
    axes_flat = axes.flatten()

    for idx, caract in enumerate(CARACTERISTICAS_EXIF):
        ax = axes_flat[idx]

        if caract not in df_exif.columns or df_exif[caract].isna().all():
            ax.text(0.5, 0.5, f'{NOMBRES_CARACTERISTICAS.get(caract, caract)}\n(Sin datos)',
                   ha='center', va='center', fontsize=10)
            ax.axis('off')
            continue

        # Histograma por cluster
        for cluster in ['facil', 'medio', 'dificil']:
            datos_cluster = df_exif[df_exif['cluster'] == cluster][caract].dropna()
            if len(datos_cluster) > 0:
                ax.hist(datos_cluster, bins=8, alpha=0.5,
                       color=COLORES_CLUSTER[cluster],
                       label=TEXTOS[f'cluster_{cluster}'],
                       edgecolor='white')

        ax.set_xlabel(NOMBRES_CARACTERISTICAS.get(caract, caract), fontsize=9)
        ax.set_ylabel('Frecuencia', fontsize=9)
        ax.legend(fontsize=7)
        ax.grid(True, alpha=0.3)

    # Ocultar axes vacios
    for idx in range(n_caract, len(axes_flat)):
        axes_flat[idx].axis('off')

    fig.suptitle(TEXTOS['titulo_exif'], fontsize=14, fontweight='bold')
    plt.tight_layout(rect=[0, 0, 1, 0.95])

    guardar_figura(fig, 'viz_29_panel_exif', output_dir, logger=logger)

    if logger:
        logger.info(f"    Caracteristicas visualizadas: {n_caract}")

    return fig


# =============================================================================
# VISUALIZACION 30: SCATTER CARACTERISTICAS VS DIFICULTAD
# =============================================================================

def generar_scatter_dificultad(
    df: pd.DataFrame,
    indice_maestro: Dict,
    output_dir: Path,
    logger=None
) -> Optional[plt.Figure]:
    """
    Genera scatter plots de caracteristicas vs IoU medio.

    Muestra la relacion entre caracteristicas fotograficas y
    la dificultad de segmentacion (medida por IoU).

    Args:
        df: DataFrame con metricas
        indice_maestro: Diccionario del indice maestro
        output_dir: Directorio de salida
        logger: Logger opcional

    Returns:
        Figura matplotlib o None
    """
    if logger:
        logger.info("  Generando scatter de dificultad...")

    col_foto = 'codigo_foto' if 'codigo_foto' in df.columns else 'foto_id'

    # Calcular dificultad
    stats = calcular_dificultad_foto(df, col_foto)

    # Extraer caracteristicas
    datos = []
    for foto_id in stats[col_foto]:
        caract = extraer_caracteristicas_exif(indice_maestro, foto_id)
        caract['foto_id'] = foto_id
        caract['cluster'] = stats[stats[col_foto] == foto_id]['cluster'].values[0]
        caract['iou_mean'] = stats[stats[col_foto] == foto_id]['iou_mean'].values[0]
        caract['iou_std'] = stats[stats[col_foto] == foto_id]['iou_std'].values[0]
        datos.append(caract)

    df_datos = pd.DataFrame(datos)

    # Convertir columnas numericas
    for col in CARACTERISTICAS_EXIF:
        if col in df_datos.columns:
            df_datos[col] = pd.to_numeric(df_datos[col], errors='coerce')

    # Seleccionar 4 caracteristicas mas relevantes
    caracteristicas_plot = ['apertura_fnumber', 'contraste_rms', 'brillo_promedio', 'nitidez_laplacian']

    fig, axes = plt.subplots(2, 2, figsize=(12, 10), squeeze=False)
    axes_flat = axes.flatten()

    for idx, caract in enumerate(caracteristicas_plot):
        ax = axes_flat[idx]

        if caract not in df_datos.columns or df_datos[caract].isna().all():
            ax.text(0.5, 0.5, f'{NOMBRES_CARACTERISTICAS.get(caract, caract)}\n(Sin datos)',
                   ha='center', va='center', fontsize=10)
            ax.axis('off')
            continue

        # Scatter por cluster
        for cluster in ['facil', 'medio', 'dificil']:
            mask = df_datos['cluster'] == cluster
            ax.scatter(df_datos.loc[mask, caract],
                      df_datos.loc[mask, 'iou_mean'],
                      c=COLORES_CLUSTER[cluster],
                      s=100, alpha=0.7,
                      label=TEXTOS[f'cluster_{cluster}'],
                      edgecolors='white', linewidth=1)

        # Linea de tendencia global
        x_valid = df_datos[caract].dropna()
        y_valid = df_datos.loc[x_valid.index, 'iou_mean']

        if len(x_valid) > 2:
            try:
                z = np.polyfit(x_valid, y_valid, 1)
                p = np.poly1d(z)
                x_line = np.linspace(x_valid.min(), x_valid.max(), 100)
                ax.plot(x_line, p(x_line), '--', color='gray', alpha=0.7, linewidth=2)

                # Calcular correlacion
                corr = np.corrcoef(x_valid, y_valid)[0, 1]
                ax.text(0.05, 0.95, f'r = {corr:.3f}', transform=ax.transAxes,
                       fontsize=10, verticalalignment='top',
                       bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
            except:
                pass

        ax.set_xlabel(NOMBRES_CARACTERISTICAS.get(caract, caract), fontsize=10)
        ax.set_ylabel('IoU Medio', fontsize=10)
        ax.legend(fontsize=8, loc='lower right')
        ax.grid(True, alpha=0.3)
        ax.set_ylim(0.6, 1.0)

    fig.suptitle(TEXTOS['titulo_scatter'], fontsize=14, fontweight='bold')
    plt.tight_layout(rect=[0, 0, 1, 0.95])

    guardar_figura(fig, 'viz_30_scatter_dificultad', output_dir, logger=logger)

    if logger:
        logger.info("    Scatter plots generados: 4")

    return fig


# =============================================================================
# VISUALIZACION 31: GALERIA ANOTADA
# =============================================================================

def generar_galeria_anotada(
    df: pd.DataFrame,
    ruta_imagenes: Path,
    output_dir: Path,
    indice_maestro: Dict = None,
    fotos_seleccionadas: List[str] = None,
    logger=None
) -> Optional[plt.Figure]:
    """
    Genera galeria anotada con fotografias seleccionadas.

    Muestra 8 fotografias representativas con sus metricas
    principales y caracteristicas.

    Args:
        df: DataFrame con metricas
        ruta_imagenes: Ruta a imagenes originales
        output_dir: Directorio de salida
        indice_maestro: Diccionario del indice maestro
        fotos_seleccionadas: Lista de foto_ids a incluir
        logger: Logger opcional

    Returns:
        Figura matplotlib o None
    """
    if logger:
        logger.info("  Generando galeria anotada...")

    col_foto = 'codigo_foto' if 'codigo_foto' in df.columns else 'foto_id'

    # Si no hay fotos seleccionadas, elegir automaticamente
    if fotos_seleccionadas is None:
        stats = calcular_dificultad_foto(df, col_foto)
        stats = stats.sort_values('iou_mean', ascending=False)

        # Seleccionar: 3 faciles, 2 medias, 3 dificiles
        faciles = stats[stats['cluster'] == 'facil'][col_foto].head(3).tolist()
        medias = stats[stats['cluster'] == 'medio'][col_foto].head(2).tolist()
        dificiles = stats[stats['cluster'] == 'dificil'][col_foto].head(3).tolist()

        fotos_seleccionadas = faciles + medias + dificiles

    n_fotos = len(fotos_seleccionadas)
    ncols = 4
    nrows = (n_fotos + ncols - 1) // ncols

    fig, axes = plt.subplots(nrows, ncols, figsize=(16, 5 * nrows), squeeze=False)
    axes_flat = axes.flatten()

    stats = calcular_dificultad_foto(df, col_foto)

    for idx, foto_id in enumerate(fotos_seleccionadas):
        ax = axes_flat[idx]

        # Cargar thumbnail
        thumbnail = cargar_imagen_thumbnail(foto_id, ruta_imagenes, size=(400, 400))

        if thumbnail is not None:
            ax.imshow(thumbnail)
        else:
            ax.set_facecolor('#f0f0f0')
            ax.text(0.5, 0.5, f'{foto_id}\n(Imagen no disponible)',
                   ha='center', va='center', fontsize=10)

        # Obtener metricas
        foto_stats = stats[stats[col_foto] == foto_id]
        if len(foto_stats) > 0:
            iou_mean = foto_stats['iou_mean'].values[0]
            iou_std = foto_stats['iou_std'].values[0]
            cluster = foto_stats['cluster'].values[0]
            color = COLORES_CLUSTER[cluster]
        else:
            iou_mean = 0
            iou_std = 0
            cluster = 'desconocido'
            color = 'gray'

        # Obtener caracteristicas EXIF
        if indice_maestro and foto_id in indice_maestro:
            caract = extraer_caracteristicas_exif(indice_maestro, foto_id)
            apertura = caract.get('apertura_fnumber', 'N/A')
            iso = caract.get('iso', 'N/A')
        else:
            apertura = 'N/A'
            iso = 'N/A'

        # Titulo con metricas
        titulo = f'{foto_id}\n'
        titulo += f'IoU: {iou_mean:.3f} (±{iou_std:.3f})\n'
        titulo += f'f/{apertura} | ISO {iso}'

        ax.set_title(titulo, fontsize=9, fontweight='bold', color=color)

        # Borde coloreado
        for spine in ax.spines.values():
            spine.set_edgecolor(color)
            spine.set_linewidth(4)

        ax.set_xticks([])
        ax.set_yticks([])

    # Ocultar axes vacios
    for idx in range(n_fotos, len(axes_flat)):
        axes_flat[idx].axis('off')

    fig.suptitle(TEXTOS['titulo_galeria'], fontsize=14, fontweight='bold')

    # Leyenda
    legend_elements = [
        Patch(facecolor=COLORES_CLUSTER['facil'], label=TEXTOS['cluster_facil']),
        Patch(facecolor=COLORES_CLUSTER['medio'], label=TEXTOS['cluster_medio']),
        Patch(facecolor=COLORES_CLUSTER['dificil'], label=TEXTOS['cluster_dificil'])
    ]
    fig.legend(handles=legend_elements, loc='lower center', ncol=3, fontsize=10,
               bbox_to_anchor=(0.5, 0.01))

    plt.tight_layout(rect=[0, 0.04, 1, 0.95])

    guardar_figura(fig, 'viz_31_galeria_anotada', output_dir, logger=logger)

    if logger:
        logger.info(f"    Fotografias en galeria: {n_fotos}")

    return fig


# =============================================================================
# FUNCION ORQUESTADORA
# =============================================================================

def ejecutar_bloque4(
    ruta_metricas: Path = None,
    ruta_imagenes: Path = None,
    ruta_indice: Path = None,
    output_dir: Path = None
) -> Dict[str, any]:
    """
    Ejecuta todas las visualizaciones del Bloque 4.

    Genera 4 visualizaciones de catalogo fotografico:
    - Mosaico por clusters
    - Panel EXIF
    - Scatter dificultad
    - Galeria anotada

    Args:
        ruta_metricas: Ruta al CSV de metricas fusionadas
        ruta_imagenes: Ruta al directorio de imagenes
        ruta_indice: Ruta al indice maestro JSON
        output_dir: Directorio de salida

    Returns:
        Diccionario con listas de figuras generadas y errores
    """
    logger = configurar_logging('Fase2E_Bloque4')

    logger.info("=" * 70)
    logger.info("FASE 2E - BLOQUE 4: CATALOGO FOTOGRAFICO")
    logger.info("=" * 70)

    aplicar_estilo()

    # Resolver rutas
    if ruta_metricas is None:
        ruta_metricas = ARCHIVOS['metricas_fusionadas']
    if ruta_imagenes is None:
        ruta_imagenes = RUTAS['imagenes']
    if ruta_indice is None:
        ruta_indice = ARCHIVOS['indice_maestro']
    if output_dir is None:
        output_dir = RUTAS.get('output_bloque4', RUTAS['output'] / 'bloque4_catalogo')

    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    logger.info(f"\nDirectorio de salida: {output_dir}")
    logger.info(f"Ruta imagenes: {ruta_imagenes}")

    # Cargar datos
    logger.info("\n" + "-" * 50)
    logger.info("Cargando datos...")

    try:
        df = pd.read_csv(ruta_metricas)
        df = agregar_columna_modelo(df)
        logger.info(f"  Metricas cargadas: {len(df)} registros")
    except Exception as e:
        logger.error(f"Error cargando metricas: {e}")
        return {'error': str(e), 'figuras': [], 'errores': []}

    # Cargar indice maestro
    indice_maestro = None
    try:
        ruta_indice = Path(ruta_indice)
        if ruta_indice.exists():
            with open(ruta_indice, 'r') as f:
                indice_maestro = json.load(f)
            logger.info(f"  Indice maestro cargado: {len(indice_maestro)} fotos")
    except Exception as e:
        logger.warning(f"  Indice maestro no disponible: {e}")

    resultados = {'figuras': [], 'errores': [], 'warnings': []}

    # =========================================================================
    # VISUALIZACION 28: MOSAICO CLUSTERS
    # =========================================================================
    logger.info("\n" + "-" * 50)
    logger.info("Generando viz_28: Mosaico por clusters...")

    try:
        fig = generar_mosaico_clusters(df, ruta_imagenes, output_dir, indice_maestro, logger)
        if fig:
            resultados['figuras'].append('viz_28_mosaico_clusters')
            plt.close(fig)
    except Exception as e:
        logger.error(f"  Error en viz_28: {e}")
        resultados['errores'].append(('viz_28_mosaico_clusters', str(e)))

    # =========================================================================
    # VISUALIZACION 29: PANEL EXIF
    # =========================================================================
    logger.info("\n" + "-" * 50)
    logger.info("Generando viz_29: Panel EXIF...")

    try:
        fig = generar_panel_exif(df, indice_maestro, output_dir, logger)
        if fig:
            resultados['figuras'].append('viz_29_panel_exif')
            plt.close(fig)
    except Exception as e:
        logger.error(f"  Error en viz_29: {e}")
        resultados['errores'].append(('viz_29_panel_exif', str(e)))

    # =========================================================================
    # VISUALIZACION 30: SCATTER DIFICULTAD
    # =========================================================================
    logger.info("\n" + "-" * 50)
    logger.info("Generando viz_30: Scatter dificultad...")

    try:
        fig = generar_scatter_dificultad(df, indice_maestro, output_dir, logger)
        if fig:
            resultados['figuras'].append('viz_30_scatter_dificultad')
            plt.close(fig)
    except Exception as e:
        logger.error(f"  Error en viz_30: {e}")
        resultados['errores'].append(('viz_30_scatter_dificultad', str(e)))

    # =========================================================================
    # VISUALIZACION 31: GALERIA ANOTADA
    # =========================================================================
    logger.info("\n" + "-" * 50)
    logger.info("Generando viz_31: Galeria anotada...")

    try:
        fig = generar_galeria_anotada(df, ruta_imagenes, output_dir, indice_maestro,
                                       FOTOS_SELECCIONADAS, logger)
        if fig:
            resultados['figuras'].append('viz_31_galeria_anotada')
            plt.close(fig)
    except Exception as e:
        logger.error(f"  Error en viz_31: {e}")
        resultados['errores'].append(('viz_31_galeria_anotada', str(e)))

    # =========================================================================
    # RESUMEN FINAL
    # =========================================================================
    logger.info("\n" + "=" * 70)
    logger.info("RESUMEN BLOQUE 4")
    logger.info("=" * 70)
    logger.info(f"Visualizaciones generadas: {len(resultados['figuras'])}/4")
    logger.info(f"Errores: {len(resultados['errores'])}")

    if resultados['figuras']:
        logger.info("\nArchivos generados:")
        for fig_name in resultados['figuras']:
            logger.info(f"  - {fig_name}.png")

    if resultados['errores']:
        logger.info("\nErrores encontrados:")
        for err_name, err_msg in resultados['errores']:
            logger.info(f"  - {err_name}: {err_msg}")

    return resultados


# =============================================================================
# PUNTO DE ENTRADA
# =============================================================================

if __name__ == "__main__":
    resultados = ejecutar_bloque4(
       ruta_metricas=Path('/content/drive/MyDrive/TFM/3_Analisis/fase2b_correlaciones/metricas_fusionadas.csv'),
    ruta_imagenes=Path('/content/drive/MyDrive/TFM/0_Imagenes'),
    ruta_indice=Path('/content/drive/MyDrive/TFM/3_Analisis/fase1_integracion/indice_maestro.json'),
    output_dir=Path('/content/drive/MyDrive/TFM/3_Analisis/fase2e_visualizaciones/bloque4_catalogo')
    )
    print(f"\nVisualizaciones generadas: {len(resultados.get('figuras', []))}/4")
    print(f"Errores: {len(resultados.get('errores', []))}")

[01:56:16] INFO - ======================================================================


INFO:Fase2E_Bloque4:======================================================================


[01:56:16] INFO - FASE 2E - BLOQUE 4: CATALOGO FOTOGRAFICO


INFO:Fase2E_Bloque4:FASE 2E - BLOQUE 4: CATALOGO FOTOGRAFICO


[01:56:16] INFO - ======================================================================


...

[01:57:11] INFO -   - viz_30_scatter_dificultad.png


INFO:Fase2E_Bloque4:  - viz_30_scatter_dificultad.png


[01:57:11] INFO -   - viz_31_galeria_anotada.png


INFO:Fase2E_Bloque4:  - viz_31_galeria_anotada.png



Visualizaciones generadas: 4/4
Errores: 0
